# AI To Play Mine Sweeper

## The Game Object

I developed the pythonic minesweeper. It is unit tested fairly throughly; up until visualizations and tracking features of the project present themselves. Note: The 9's are bombs; because it is impossible for a cell to be sourounded by more Bombs then there are cells surronding it.

In [39]:
import game
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from ipywidgets import IntProgress
from IPython.display import display

In [41]:
from importlib import reload

reload(game)

<module 'game' from '/home/lukenelson/dev/MineSweeperAI/game.py'>

In [3]:
mygame = game.minesweeper(14, 8, 25)


for row in mygame.board:
    for c in row:
        print(c, "\t", end="")
    print("\n")
#mygame.clickCell((0,0))

1.0 	2.0 	1.0 	1.0 	1.0 	2.0 	2.0 	1.0 	1.0 	9.0 	1.0 	1.0 	9.0 	2.0 	

9.0 	2.0 	9.0 	1.0 	1.0 	9.0 	9.0 	2.0 	3.0 	3.0 	2.0 	2.0 	3.0 	9.0 	

2.0 	3.0 	2.0 	1.0 	1.0 	3.0 	3.0 	3.0 	9.0 	9.0 	1.0 	1.0 	9.0 	2.0 	

1.0 	9.0 	1.0 	0.0 	0.0 	1.0 	9.0 	2.0 	2.0 	2.0 	1.0 	2.0 	3.0 	3.0 	

2.0 	2.0 	2.0 	2.0 	2.0 	2.0 	2.0 	2.0 	1.0 	0.0 	0.0 	1.0 	9.0 	9.0 	

9.0 	1.0 	1.0 	9.0 	9.0 	1.0 	1.0 	9.0 	2.0 	2.0 	2.0 	3.0 	4.0 	4.0 	

1.0 	1.0 	1.0 	2.0 	2.0 	1.0 	1.0 	1.0 	3.0 	9.0 	9.0 	3.0 	9.0 	9.0 	

0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	0.0 	2.0 	9.0 	9.0 	3.0 	3.0 	9.0 	



## Visualizing the Game

The following is a rough gui for the game. Feel free to mess around; when you loose all of the cells will disable and you will be alerted. When you win you will be alerted. Note: If you want to start a new game you will have to run the cell above because you need to make a new object to start over

In [4]:
mygame.playOnNoteBook()

## Neruel Network to Play Mine Sweeper (Classification Approach) 

We will attempt to play minesweeper by generating lots of games (which will be used as our data). 

Things that we'll need to do:

Pad the game board with some element.
This way we distiguish the edges of the game board (all of our data needs to have the same number of features) 

Represent "unknown squares" in our classification we will have some data that are "completed games" (the whole board is visible) and others where the game is 80% finished; 60%; 40% and 20%. (Our "Solve Percent" utility will help us with this)

In [5]:
mygame = game.minesweeper(14, 8, 25)
mygame.solvePercent(.2)
mygame.playOnNoteBook()

In [105]:
theData = []
N = 2 #N represents the number of surronding squares to get


for p in [.8, .6, .4, .2]:
    for i in range(1000):
        mygame = game.minesweeper(10, 10, 23)
        mygame.solvePercent(p)
        dataToAdd = np.pad(mygame.visible, (N), 'constant', constant_values=(-1))
        dataToAdd = np.where(dataToAdd=='*', 0, dataToAdd)
        theData.append(dataToAdd)
        
for i in range(1000):
    mygame = game.minesweeper(10, 10, 23)
    dataToAdd = np.pad(mygame.board, (N), 'constant', constant_values=(-1))
    theData.append(dataToAdd.astype('<U3'))


In [106]:
theRealData = []


for data in theData:
    innerData = data[N:-N, N:-N]
    for index, x in np.ndenumerate(innerData):
        toAppend = []
        toAppend.append(x)
        for i in range((2*N+1)**2):
            if i != (((2*N+1)**2)//2):
                a = i//(2*N+1)
                b = i%(2*N+1)
                toAppend.append(data[index[0] + a, index[1] + b])
        #toAppend.append(data[index[0], index[1]])
        #toAppend.append(data[index[0], index[1]+1])
        #toAppend.append(data[index[0], index[1]+2])
        #toAppend.append(data[index[0], index[1]+3])
        #toAppend.append(data[index[0], index[1]+4])
        #toAppend.append(data[index[0]+1, index[1]])
        #toAppend.append(data[index[0]+1, index[1]+1])
        #toAppend.append(data[index[0]+1, index[1]+2])
        #toAppend.append(data[index[0]+1, index[1]+3])
        #toAppend.append(data[index[0]+1, index[1]+4])
        #toAppend.append(data[index[0]+2, index[1]])
        #toAppend.append(data[index[0]+2, index[1]+1])
        #toAppend.append(data[index[0]+2, index[1]+3])
        #toAppend.append(data[index[0]+2, index[1]+4])
        #toAppend.append(data[index[0]+3, index[1]])
        #toAppend.append(data[index[0]+3, index[1]+1])
        #toAppend.append(data[index[0]+3, index[1]+2])
        #toAppend.append(data[index[0]+3, index[1]+3])
        #toAppend.append(data[index[0]+3, index[1]+4])
        #toAppend.append(data[index[0]+4, index[1]])
        #toAppend.append(data[index[0]+4, index[1]+1])
        #toAppend.append(data[index[0]+4, index[1]+2])
        #toAppend.append(data[index[0]+4, index[1]+3])
        #toAppend.append(data[index[0]+4, index[1]+4])
        theRealData.append(toAppend)

(Comment this line out if you don't want to eliminate the unknown value prediction)

In [107]:
theRealData = np.array(theRealData)
theRealData = theRealData[theRealData[:, 0] != '?']

This next line will make it so that it's classifying not bomb (which is true) and bomb (false) -- so if a cell is a bomb it will give it a false value. Think of it as an "okay to click" classification.

In [108]:
theRealData[:, 0] = ['1.0' if r else '0.0' for r in (theRealData[:, 0] != '9.0')]
theRealData

array([['1.0', '-1', '-1', ..., '?', '?', '?'],
       ['1.0', '-1', '-1', ..., '?', '-1', '-1'],
       ['1.0', '?', '?', ..., '?', '?', '?'],
       ...,
       ['1.0', '1.0', '0.0', ..., '-1.', '-1.', '-1.'],
       ['0.0', '0.0', '0.0', ..., '-1.', '-1.', '-1.'],
       ['1.0', '0.0', '1.0', ..., '-1.', '-1.', '-1.']], dtype='<U3')

In [109]:
theRealData = np.where(theRealData=='?', 10, theRealData)
X, y = theRealData[:, 1:].astype(float).astype(str), theRealData[:, 0].astype(float).astype(str)

enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(X)

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)

In [110]:
#print([cat.shape for cat in enc.categories_])
print(enc.categories_)
X = enc.transform(X).toarray()

[array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.0', '1.0', '10.0', '2.0', '3.0', '4.0', '5.0', '6.0',
       '7.0', '9.0'], dtype='<U32'), array(['-1.0', '0.

In [111]:
X.shape

(309234, 264)

In [112]:
y.shape

(309234,)

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

The Hidden Layer "24" is inspired by the idea that each surrounding node has been one-hot-encoded into 12 features

In [159]:
tolList = [.01, .005, .0025]
hidden_layer_sizes = [50, 60, 70, 80, 90, 100]
activationList = ['identity', 'logistic', 'tanh', 'relu']
alphaList = [1e-3, 1e-4, 1e-5, 1e-6]

In [167]:

clf = MLPClassifier(solver='adam', activation="tanh", alpha=1e-5, hidden_layer_sizes=(100), verbose=True, max_iter=300, tol=.01)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

Iteration 1, loss = 0.20922322
Iteration 2, loss = 0.16853829
Iteration 3, loss = 0.15110644
Iteration 4, loss = 0.13380831
Iteration 5, loss = 0.12000269
Iteration 6, loss = 0.11141227
Iteration 7, loss = 0.10558043
Iteration 8, loss = 0.10134970
Iteration 9, loss = 0.09780280
Iteration 10, loss = 0.09475220
Iteration 11, loss = 0.09224368
Iteration 12, loss = 0.08969833
Iteration 13, loss = 0.08763518
Iteration 14, loss = 0.08565129
Iteration 15, loss = 0.08362612
Iteration 16, loss = 0.08216856
Training loss did not improve more than tol=0.010000 for 10 consecutive epochs. Stopping.
0.9568634368140483


In [157]:
for layerSize in hidden_layer_sizes:
    clf = MLPClassifier(solver='adam', 
                        activation="tanh", 
                        alpha=1e-5, 
                        hidden_layer_sizes=(layerSize), 
                        max_iter=300, 
                        tol=0.01)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print("LayerSize {}; accuracy: {}; False-Negative: {}".format(layerSize, accuracy_score(y_test, y_pred), confusion_matrix(y_test, y_pred)[0][1]))

LayerSize 50; accuracy: 0.9570790216368767; False-Negative: 3300
LayerSize 60; accuracy: 0.9575395892129194; False-Negative: 3280
LayerSize 70; accuracy: 0.9577159767952336; False-Negative: 3120
LayerSize 80; accuracy: 0.957343603010348; False-Negative: 2901
LayerSize 90; accuracy: 0.9574611947318908; False-Negative: 2742
LayerSize 100; accuracy: 0.9578825650674192; False-Negative: 3294


In [158]:
for layerSize in [85, 87, 90, 93, 95]:
    clf = MLPClassifier(solver='adam', 
                        activation="tanh", 
                        alpha=1e-5, 
                        hidden_layer_sizes=(layerSize), 
                        max_iter=300, 
                        tol=0.01)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print("LayerSize {}; accuracy: {}; False-Negative: {}".format(layerSize, accuracy_score(y_test, y_pred), confusion_matrix(y_test, y_pred)[0][1]))

LayerSize 85; accuracy: 0.9562754782063343; False-Negative: 2687
LayerSize 87; accuracy: 0.9576179836939479; False-Negative: 3255
LayerSize 90; accuracy: 0.9576963781749764; False-Negative: 3447
LayerSize 93; accuracy: 0.957931561618062; False-Negative: 3161
LayerSize 95; accuracy: 0.9573534023204766; False-Negative: 3035


In [160]:
for activationTest in activationList:
    clf = MLPClassifier(solver='adam', 
                        activation=activationTest, 
                        alpha=1e-5, 
                        hidden_layer_sizes=(80), 
                        max_iter=300, 
                        tol=0.01)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print("Activation {}; accuracy: {}; False-Negative: {}".format(activationTest, accuracy_score(y_test, y_pred), confusion_matrix(y_test, y_pred)[0][1]))

Activation identity; accuracy: 0.9303857008466604; False-Negative: 4859
Activation logistic; accuracy: 0.9522381624333647; False-Negative: 3649
Activation tanh; accuracy: 0.9582157416117906; False-Negative: 2957
Activation relu; accuracy: 0.9562754782063343; False-Negative: 2994


In [161]:
for activationTest in ["tanh", "relu", "tanh", "relu", "tanh", "relu", "tanh", "relu"]:
    clf = MLPClassifier(solver='adam', 
                        activation=activationTest, 
                        alpha=1e-5, 
                        hidden_layer_sizes=(80), 
                        max_iter=300, 
                        tol=0.01)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print("Activation {}; accuracy: {}; False-Negative: {}".format(activationTest, accuracy_score(y_test, y_pred), confusion_matrix(y_test, y_pred)[0][1]))

Activation tanh; accuracy: 0.957480793352148; False-Negative: 2946
Activation relu; accuracy: 0.954560598933835; False-Negative: 2756
Activation tanh; accuracy: 0.9581863436814049; False-Negative: 3156
Activation relu; accuracy: 0.9563832706177485; False-Negative: 3203
Activation tanh; accuracy: 0.9579511602383193; False-Negative: 3050
Activation relu; accuracy: 0.9548447789275635; False-Negative: 2700
Activation tanh; accuracy: 0.9577845719661335; False-Negative: 3078
Activation relu; accuracy: 0.9553837409846346; False-Negative: 2922


In [162]:
for alphaTest in alphaList:
    clf = MLPClassifier(solver='adam', 
                        activation="relu", 
                        alpha=alphaTest, 
                        hidden_layer_sizes=(80), 
                        max_iter=300, 
                        tol=0.01)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print("Alpha {}; accuracy: {}; False-Negative: {}".format(alphaTest, accuracy_score(y_test, y_pred), confusion_matrix(y_test, y_pred)[0][1]))

Alpha 0.001; accuracy: 0.9560892913138915; False-Negative: 2912
Alpha 0.0001; accuracy: 0.955648322358106; False-Negative: 2906
Alpha 1e-05; accuracy: 0.9569418312950768; False-Negative: 3038
Alpha 1e-06; accuracy: 0.956961429915334; False-Negative: 3011


In [164]:
clf = MLPClassifier(solver='adam', 
                    activation="relu", 
                    alpha=1e-4, 
                    hidden_layer_sizes=(80), 
                    max_iter=300, 
                    tol=0.01)

clf.fit(X_train, y_train)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.956961429915334
[[14180  3011]
 [ 1381 83476]]


In [165]:
clf.predict_proba(X_test[0].reshape(1, -1))[0][1]

0.9964861680032117

# The AI that Plays Mine Sweeper

In [ ]:
wins = 0
loses = 0
gamesToPlay = 1000
bombGuessTol = .85

f = IntProgress(min=0, max=gamesToPlay) # instantiate the bar
display(f) # display the bar

for _ in range(gamesToPlay):
    f.value += 1
    mygame = game.minesweeper(9, 9, 10, record=True)
    #mygame.playOnNoteBook()

    #Our algorithm selects (0,0) first every time any way -- input is all the same on all unknown...
    #To test if there is a better place than this think about classifying better starting points... Differn't shapes...
    mygame.clickCell((1,1))#gameboard[1][1].click()

    while ("?" in " ".join([" ".join(map(str, row)) for row in mygame.visible])):
        curbest = 0
        bestindex = (0, 0)
        data = np.array(mygame.visible).astype('<U3')
        data = np.pad(data, (N), 'constant', constant_values=(-1))
        data = np.where(data=='*', 0, data)
        max_right = max(np.where(np.array(mygame.visible) != '?')[0])+1 
        max_down = max(np.where(np.array(mygame.visible) != '?')[1])+1
        innerData = (data[N:-N, N:-N])[0:max_right+1, 0:max_down+1] 
        for index, x in np.ndenumerate(innerData):
            if x == "?":
                toAppend = []
                for i in range((2*N+1)**2):
                    if i != (((2*N+1)**2)//2):
                        a = i//(2*N+1)
                        b = i%(2*N+1)
                        toAppend.append(data[index[0] + a, index[1] + b])
                toAppend = np.array(toAppend)
                toAppend = np.where(toAppend=='?', 10, toAppend)
                toAppend = toAppend.astype(float).astype(str)
                toAppend = toAppend.reshape(1,-1)
                toAppend = enc.transform(toAppend).toarray()
                if clf.predict_proba(toAppend)[0][1] > curbest:
                    curbest = clf.predict_proba(toAppend)[0][1]
                    bestindex = index
                if clf.predict_proba(toAppend)[0][0] > bombGuessTol:
                    #print(index)
                    mygame.visible[index[0]][index[1]] = 9.0
                    #mygame.gameboard[index[0]][index[1]].description = "9.0"
        #print(curbest)
        #print(bestindex)
        mygame.clickCell(bestindex)
        #mygame.gameboard[bestindex[0]][bestindex[1]].click()
        #mygame.clickCell(bestindx)

        #print(c, bestindex)
    if mygame.visible == "You have lost":
        loses += 1
    else:
        wins += 1

print("The AI won {} times and lose {} times".format(wins, loses))        
    
#print(mygame.visible)

IntProgress(value=0, max=1000)

In [33]:
mygame.visible


[['*', '*', '*', '*', '*', '*', '*', '*', '*'],
 ['*', '*', '*', '*', '*', '*', '*', '1.0', '1.0'],
 ['*', '*', '*', '*', '*', '*', '*', '1.0', '?'],
 ['2.0', '2.0', '1.0', '*', '*', '1.0', '1.0', '3.0', '?'],
 ['?', '?', '1.0', '*', '*', '1.0', '?', '?', '?'],
 ['?', '?', '1.0', '1.0', '1.0', '2.0', '?', '?', '?'],
 ['?', '?', '?', '?', '?', '?', '?', '?', '?'],
 ['?', '?', '?', '?', '?', '?', '?', '?', '?'],
 ['?', '?', '?', '?', '?', '?', '?', '?', '?']]

# Idea's to Improve:
### Mess with Hyper Params
### Mess with the solve function (make exact percentages etc)
### Mess with what we are classifying (We just want to know if something is a bomb or not...
### More Data surronding each point

# Messing with Hyper Params

In [ ]:
#clf2 = MLPClassifier(solver='lbfgs', alpha=1e-5, random_state=1)

#clf2.fit(X_train, y_train)

#y_pred = clf2.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [ ]:
# This one is inspired by the one hot incoding (Each feature was split into 11 parts)
#clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(11), random_state=1)

#clf3.fit(X_train, y_train)

#y_pred = clf3.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [ ]:
#import pickle

#filename = 'clf1.sav'
#pickle.dump(clf, open(filename, 'wb'))

#filename = 'clf2.sav'
#pickle.dump(clf2, open(filename, 'wb'))

#filename = 'clf3.sav'
#pickle.dump(clf3, open(filename, 'wb'))

In [ ]:
#clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(8, 3), random_state=1)

#clf3.fit(X_train, y_train)

#y_pred = clf3.predict(X_test)

#print(accuracy_score(y_test, y_pred))

In [ ]:
#filename = 'clf3.sav'
#pickle.dump(clf3, open(filename, 'wb'))

# Messing with what we are classifying

In [ ]:
np.unique(y.shape)